----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Content layer - Pricing stream - Used as a real-time data cache
This notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) either directly from the LSEG Data Platform or via LSEG Workspace or CodeBook. The example shows how to define a Pricing stream object, which automatically manages a streaming cache available for access at any time. Your application can then reach into this cache and pull out real-time snapshots as Pandas DataFrames by just calling a simple access method.

Using a Pricing stream object that way prevents your application from sending too many requests to the platform. This is particularly useful if your application needs to retrieve real-time snapshots at regular and short intervals.

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) to the LSEG Developer Community portal you will get free access to a number of learning materials 
 on the [LSEG Developer Portal](https://developers.lseg.com/en).

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on 
the [LSEG Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help.

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the LSEG Data Library in the _lseg-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _LD_LIB_CONFIG_PATH_ environment variable. The _LD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [20]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [21]:
import lseg.data as ld
from lseg.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [22]:
ld.open_session()

<lseg.data.session.Definition object at 0x1b056c8be90 {name='workspace'}>

## Retrieve data

### Create and open a Pricing stream object

The Pricing stream object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, the Pricing stream will retrieve all fields available for the requested instruments

In [23]:
stream = ld.content.pricing.Definition(
    universe = ['EUR=', 'GBP=', 'JPY=', 'CAD='], 
    fields = ['BID', 'ASK']
).get_stream()

The open method tells the Pricing stream object to subscribe to the streams of the requested instruments.

In [24]:
stream.open()

<OpenState.Opened: 'Opened'>

As soon as the open method returns, the stream object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from Eikon / LSEG Workspace. All this happens behind the scene, waiting for your application to pull out data from the cache.  

### Extract snapshot data from the streaming cache
Once the stream is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot can be called any number of times. As these calls return the latest received values, successive calls to get_snapshot may return different values. Returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh values. 

In [25]:
df = stream.get_snapshot()
display(df)

Instrument     ASK     BID
0       EUR=   1.085  1.0846
1       GBP=  1.2837  1.2836
2       JPY=  155.84  155.79
3       CAD=  1.3698  1.3693

### Get a snapshot for a subset of instruments and fields

In [26]:
df = stream.get_snapshot(
    universe = ['EUR=', 'GBP='], 
    fields = ['BID', 'ASK']
)
display(df)

Instrument     BID     ASK
0       EUR=  1.0846   1.085
1       GBP=  1.2836  1.2837

### Other options to get values from the streaming cache

#### Direct access to real-time fields

In [27]:
print('GBP/BID:', stream['GBP=']['BID'])
print('EUR/BID:', stream['EUR=']['BID'])

GBP/BID: 1.2836
EUR/BID: 1.0846


#### Direct acces to a streaming instrument

In [28]:
gbp = stream['GBP=']
print(gbp['BID'])

1.2836


#### Iterate on fields

In [29]:
print('GBP=')
for field_name, field_value in stream['GBP=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in stream['JPY=']:
    print('\t' + field_name + ': ', field_value)

GBP=
	ASK:  1.2837
	BID:  1.2836
JPY=
	ASK:  155.84
	BID:  155.79


#### Iterate on streaming instruments and fields

In [30]:
for streaming_instrument in stream:
    print(streaming_instrument.name)
    for field_name, field_value in streaming_instrument:
        print('\t' + field_name + ': ', field_value)

EUR=
	ASK:  1.085
	BID:  1.0846
GBP=
	ASK:  1.2837
	BID:  1.2836
JPY=
	ASK:  155.84
	BID:  155.79
CAD=
	ASK:  1.3698
	BID:  1.3693


### Close the stream

In [31]:
stream.close()

<OpenState.Closed: 'Closed'>

Once closed is called the Pricing stream object stops updating its internal cache. The get_snapshot function can still be called but after the close it always return the same values. 

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [32]:
mixed = ld.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()

mixed.open()
mixed.get_snapshot()

Instrument     ASK     BID
0       EUR=   1.085  1.0846
1       GBP=  1.2838  1.2834
2       JPY=  155.84  155.79
3       CAD=  1.3698  1.3694
4     BADRIC    <NA>    <NA>

You can check the Status of any instrument, so lets check the invalid one

In [34]:
display(mixed['BADRIC'].code)
display(mixed['BADRIC'].message)

'Closed'

'The record could not be found'

As you will note, for an invalid instrument we get:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   

However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: The exact wording of **message** can change over time - therefore,only use the **code** value for any programmatic decision making.

In [17]:
mixed.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [18]:
ld.close_session()